In [77]:
import pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture as GMM
from FeatureExtraction import extract_features
#from speakerfeatures import extract_features
import warnings
import os
import sys
import time
import glob
warnings.filterwarnings("ignore")

# Study's dataset

In [78]:
time1 = time.time()
training = True
dest = "2-gmm_models_scale_test_dataset/"

training_size = 8

test = "normalized"

subset = "training"
audios = f"../../dataset/{test}_dataset/{subset}/"

directories = os.listdir(audios)
print(directories)



for directory in directories:
    # 1 speaker
    # directory = directories[0]

    # List all files in the directory
    file_paths = os.listdir(audios + directory)
    file_paths = sorted(file_paths)
    file_paths = file_paths[:training_size]
    # print(file_paths)
    # Extracting features for each speaker
    features = np.asarray(())
    for path in file_paths:
        path = directory + "/" + path    
        path = path.strip()   
        print (path)
        
        # read the audio
        sr,audio = read(audios+path)
        
        # extract 40 dimensional MFCC & delta MFCC features
        vector   = extract_features(audio,sr)
        
        if features.size == 0:
            features = vector
        else:
            features = np.vstack((features, vector))
        print(features.shape)
        # when features of 5 files of speaker are concatenated, then do model training
        # -> if count == 5: --> edited below

        # speaker_dir = mfcc_dir + directory
        # speaker_name = path.split("/")[1].split(".")[0]

        # if not os.path.exists(speaker_dir):
        #     os.makedirs(speaker_dir)
        # # Assuming 'features' is the 2D array containing the features
        # csv_path = speaker_dir + "/" + speaker_name + ".csv"
        # np.savetxt(csv_path, vector, delimiter=",")
        # print("Features saved to", csv_path)
        

    if training and subset == "training":            
        gmm = GMM(n_components = 5, covariance_type='diag',n_init = 3)
        gmm.fit(features)
        # dumping the trained gaussian model
        picklefile = directory + ".gmm"
        print(picklefile)
        cPickle.dump(gmm,open(dest + picklefile,'wb'))
        print ('+ modeling completed for speaker:',picklefile," with data point = ",features.shape)   
        features = np.asarray(())
print("Total Time taken: ", round(time.time() - time1, 2), "seconds")


['Sherif', 'Youssef', 'Abdelrahman', 'Omar', 'Reem', 'Renad']
Sherif/S_1.wav
(299, 22)
Sherif/S_10.wav
(598, 22)
Sherif/S_11.wav
(897, 22)
Sherif/S_12.wav
(1196, 22)
Sherif/S_13.wav
(1495, 22)
Sherif/S_14.wav
(1794, 22)
Sherif/S_15.wav
(2093, 22)
Sherif/S_16.wav
(2392, 22)
Sherif.gmm
+ modeling completed for speaker: Sherif.gmm  with data point =  (2392, 22)
Youssef/Y_1.wav
(299, 22)
Youssef/Y_10.wav
(598, 22)
Youssef/Y_11.wav
(897, 22)
Youssef/Y_12.wav
(1196, 22)
Youssef/Y_13.wav
(1495, 22)
Youssef/Y_14.wav
(1794, 22)
Youssef/Y_15.wav
(2093, 22)
Youssef/Y_16.wav
(2392, 22)
Youssef.gmm
+ modeling completed for speaker: Youssef.gmm  with data point =  (2392, 22)
Abdelrahman/A_1.wav
(299, 22)
Abdelrahman/A_10.wav
(598, 22)
Abdelrahman/A_11.wav
(897, 22)
Abdelrahman/A_12.wav
(1196, 22)
Abdelrahman/A_13.wav
(1495, 22)
Abdelrahman/A_14.wav
(1794, 22)
Abdelrahman/A_15.wav
(2093, 22)
Abdelrahman/A_16.wav
(2392, 22)
Abdelrahman.gmm
+ modeling completed for speaker: Abdelrahman.gmm  with data p

In [79]:
test = "normalized"
subset = "testing_same"

# test = "normalized"
# subset = "testing_different"

source = f"../../dataset/{test}_dataset/{subset}/"

#path where training speakers will be saved
modelpath = dest

gmm_files = [os.path.join(modelpath,fname) for fname in 
              os.listdir(modelpath) if fname.endswith('.gmm')]

#Load the Gaussian Models
models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
print("Loaded Models: ", len(models))
speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
              in gmm_files]

error = 0
total_samples = 0.0

external_class_true_count = 0
# print("Press '1' for checking a single Audio or Press '0' for testing a complete set of audio with Accuracy?")
# take=int(input().strip())

predictions = []
gender_predictions = []
gender_gt = []
y_true = []
preprocessingMisPredictions = 0
print("Testing the model with all the samples...")
false_predictions = []
test_file = "Testing_audio_Path.txt"       
# file_paths = open(test_file,'r')
file_paths = glob.glob(f"{source}/**/*.wav", recursive=True)

# print(file_paths)

# Read the test directory and get the list of test audio files 
avg_time = 0
total_time = 0
for path in file_paths:
    time1 = time.time()   
    total_samples+= 1.0
    path=path.strip()
    # print("Testing Audio : ", path)
    sr,audio = read(path)
    vector   = extract_features(audio,sr)
    # print(vector.shape)
    log_likelihood = np.zeros(len(models)) 
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
    winner=np.argmax(log_likelihood)
    winner_score = log_likelihood[winner]
    predicted_speaker = speakers[winner]
    speaker_label = path.split("/")[-2]
    # print("OK score: ", winner_score)
    predictions.append(predicted_speaker)
    y_true.append(speaker_label)

    if predicted_speaker != speaker_label:
        print(f"{speaker_label}:{predicted_speaker}, ", end="")
        error += 1
        false_predictions.append(f"{speaker_label}:{predicted_speaker} ({path.split('/')[-1]})")
    # time.sleep(1.0)
    sample_time = time.time() - time1
    total_time += sample_time
print (error, total_samples)
accuracy = ((total_samples - error) / total_samples)

print ("Accuracy: ", round(accuracy, 4))
print(f"Average time taken per sample in ms", round((total_time/total_samples)*1000, 2))
print("Preprocessing MisPredictions: ", preprocessingMisPredictions)
# print ("The following Predictions were False :")

print(modelpath)
print ("Speaker Identified Successfully")
with open("scalibility_test.txt", "w") as f:
    print("Study's dataset false predictions (True: False)", file=f)
    for prediction in false_predictions:
        print(prediction, file=f)
    print("Accuracy: ", round(accuracy, 4), file=f)
    print(f"Average time taken per sample in ms", round((total_time/total_samples)*1000, 2), file=f, end="\n\n")


Loaded Models:  6
Testing the model with all the samples...
Renad:Reem, Renad:Reem, 2 120.0
Accuracy:  0.9833
Average time taken per sample in ms 12.82
Preprocessing MisPredictions:  0
2-gmm_models_scale_test_dataset/
Speaker Identified Successfully


In [80]:
test = "normalized"
subset = "testing_different"

# test = "normalized"
# subset = "testing_different"

source = f"../../dataset/{test}_dataset/{subset}/"

#path where training speakers will be saved
modelpath = dest

gmm_files = [os.path.join(modelpath,fname) for fname in 
              os.listdir(modelpath) if fname.endswith('.gmm')]

#Load the Gaussian Models
models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
print("Loaded Models: ", len(models))
speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
              in gmm_files]

error = 0
total_samples = 0.0

external_class_true_count = 0
# print("Press '1' for checking a single Audio or Press '0' for testing a complete set of audio with Accuracy?")
# take=int(input().strip())

predictions = []
gender_predictions = []
gender_gt = []
y_true = []
preprocessingMisPredictions = 0
print("Testing the model with all the samples...")
false_predictions = []
test_file = "Testing_audio_Path.txt"       
# file_paths = open(test_file,'r')
file_paths = glob.glob(f"{source}/**/*.wav", recursive=True)

# print(file_paths)

# Read the test directory and get the list of test audio files 
avg_time = 0
total_time = 0
for path in file_paths:
    time1 = time.time()   
    total_samples+= 1.0
    path=path.strip()
    # print("Testing Audio : ", path)
    sr,audio = read(path)
    vector   = extract_features(audio,sr)
    # print(vector.shape)
    log_likelihood = np.zeros(len(models)) 
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
    winner=np.argmax(log_likelihood)
    winner_score = log_likelihood[winner]
    predicted_speaker = speakers[winner]
    speaker_label = path.split("/")[-2]
    # print("OK score: ", winner_score)
    predictions.append(predicted_speaker)
    y_true.append(speaker_label)

    if predicted_speaker != speaker_label:
        print(f"{speaker_label}:{predicted_speaker}, ", end="")
        error += 1
        false_predictions.append(f"{speaker_label}:{predicted_speaker} ({path.split('/')[-1]})")
    # time.sleep(1.0)
    sample_time = time.time() - time1
    total_time += sample_time
print (error, total_samples)
accuracy = ((total_samples - error) / total_samples)

print ("Accuracy: ", round(accuracy, 4))
print(f"Average time taken per sample in ms", round((total_time/total_samples)*1000, 2))
print("Preprocessing MisPredictions: ", preprocessingMisPredictions)
# print ("The following Predictions were False :")

print(modelpath)
print ("Speaker Identified Successfully")


Loaded Models:  6
Testing the model with all the samples...
Sherif:Youssef, Sherif:Reem, Sherif:Abdelrahman, Sherif:Abdelrahman, Sherif:Reem, Sherif:Reem, Sherif:Reem, Sherif:Reem, Abdelrahman:Youssef, Abdelrahman:Omar, Abdelrahman:Reem, Omar:Sherif, Reem:Renad, Renad:Reem, Renad:Reem, Renad:Reem, 16 48.0
Accuracy:  0.6667
Average time taken per sample in ms 13.12
Preprocessing MisPredictions:  0
2-gmm_models_scale_test_dataset/
Speaker Identified Successfully


# VidTIMIT

In [81]:
time1 = time.time()
training = True
dest = "2-gmm_models_scale_test_VidTIMIT/"

training_size = 8

test = "normalized"

subset = "training"
audios = f"../../dataset/{test}_dataset/VidTIMIT_{subset}/"

directories = os.listdir(audios)
print(directories)



for directory in directories:
    # 1 speaker
    # directory = directories[0]

    # List all files in the directory
    file_paths = os.listdir(audios + directory)
    file_paths = sorted(file_paths)
    file_paths = file_paths[:training_size]
    # print(file_paths)
    # Extracting features for each speaker
    features = np.asarray(())
    for path in file_paths:
        path = directory + "/" + path    
        path = path.strip()   
        print (path)
        
        # read the audio
        sr,audio = read(audios+path)
        
        # extract 40 dimensional MFCC & delta MFCC features
        vector   = extract_features(audio,sr)
        
        if features.size == 0:
            features = vector
        else:
            features = np.vstack((features, vector))
        # print(features.shape)


    if training and subset == "training":            
        gmm = GMM(n_components = 5, covariance_type='diag',n_init = 3)
        gmm.fit(features)
        # dumping the trained gaussian model
        picklefile = directory + ".gmm"
        print(picklefile)
        cPickle.dump(gmm,open(dest + picklefile,'wb'))
        print ('+ modeling completed for speaker:',picklefile," with data point = ",features.shape)   
        features = np.asarray(())
print("Total Time taken: ", round(time.time() - time1, 2), "seconds")


['fdms0-011', 'fjem0-070', 'fdac1-021', 'mrgg0-050', 'mpdf0-060', 'fpkt0-084', 'fcft0-030', 'fcmh0-078', 'mbdg0-079', 'mjar0-031', 'fkms0-062', 'mmdm2-087', 'mcem0-044', 'mtmr0-066', 'mccs0-007', 'fdrd1-056', 'fedw0-082', 'fadg0-016', 'fgjd0-059', 'fjas0-074', 'msjs1-002', 'mjsw0-054', 'mrcz0-068', 'faks0-076', 'mdab0-023', 'fjwb0-080', 'mbjk0-020', 'mabw0-043', 'mpgl0-048', 'mgwt0-017', 'mtas1-026', 'fram1-035', 'fjre0-051', 'mdld0-018', 'fcrh0-027', 'mreb0-061', 'mrjo0-008', 'fcmr0-045', 'felc0-083', 'mmdb1-013', 'mwbt0-001', 'mstk0-064', 'mdbb0-041']
fdms0-011/sa1.wav
fdms0-011/sa2.wav
fdms0-011/si1848.wav
fdms0-011/sx138.wav
fdms0-011/sx228.wav
fdms0-011/sx318.wav
fdms0-011/sx408.wav
fdms0-011/sx48.wav
fdms0-011.gmm
+ modeling completed for speaker: fdms0-011.gmm  with data point =  (3184, 22)
fjem0-070/sa1.wav
fjem0-070/sa2.wav
fjem0-070/si1264.wav
fjem0-070/si1894.wav
fjem0-070/sx184.wav
fjem0-070/sx274.wav
fjem0-070/sx364.wav
fjem0-070/sx94.wav
fjem0-070.gmm
+ modeling completed

In [82]:
test = "normalized"
subset = "VidTIMIT_testing"

# test = "normalized"
# subset = "testing_different"

source = f"../../dataset/{test}_dataset/{subset}/"

#path where training speakers will be saved
modelpath = dest

gmm_files = [os.path.join(modelpath,fname) for fname in 
              os.listdir(modelpath) if fname.endswith('.gmm')]

#Load the Gaussian Models
models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
print("Loaded Models: ", len(models))
speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
              in gmm_files]

error = 0
total_samples = 0.0

external_class_true_count = 0
# print("Press '1' for checking a single Audio or Press '0' for testing a complete set of audio with Accuracy?")
# take=int(input().strip())

predictions = []
gender_predictions = []
gender_gt = []
y_true = []
preprocessingMisPredictions = 0
print("Testing the model with all the samples...")
false_predictions = []
test_file = "Testing_audio_Path.txt"       
# file_paths = open(test_file,'r')
file_paths = glob.glob(f"{source}/**/*.wav", recursive=True)

# Read the test directory and get the list of test audio files 
avg_time = 0
total_time = 0
for path in file_paths:
    time1 = time.time()   
    total_samples+= 1.0
    path=path.strip()
    # print("Testing Audio : ", path)
    sr,audio = read(path)
    vector   = extract_features(audio,sr)
    # print(vector.shape)
    log_likelihood = np.zeros(len(models)) 
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
    winner=np.argmax(log_likelihood)
    winner_score = log_likelihood[winner]
    predicted_speaker = speakers[winner]
    speaker_label = path.split("/")[-2]
    # print("OK score: ", winner_score)
    predictions.append(predicted_speaker)
    y_true.append(speaker_label)

    if predicted_speaker != speaker_label:
        print(f"{speaker_label}:{predicted_speaker}, ", end="")
        error += 1
        false_predictions.append(f"{speaker_label}:{predicted_speaker} ({path.split('/')[-1]})")
    # time.sleep(1.0)
    sample_time = time.time() - time1
    total_time += sample_time
print (error, total_samples)
accuracy = ((total_samples - error) / total_samples)

print ("Accuracy: ", round(accuracy, 4))
print(f"Average time taken per sample in ms", round((total_time/total_samples)*1000, 2))
print("Preprocessing MisPredictions: ", preprocessingMisPredictions)
# print ("The following Predictions were False :")

print(modelpath)
print ("Speaker Identified Successfully")
with open("scalibility_test.txt", "a") as f:
    print("VidTIMIT false predictions (True: False)", file=f)
    for prediction in sorted(false_predictions):
        print(prediction, file=f)
    print("Accuracy: ", round(accuracy, 4), file=f)
    print(f"Average time taken per sample in ms", round((total_time/total_samples)*1000, 2), file=f, end="\n\n")


Loaded Models:  43
Testing the model with all the samples...
fjem0-070:mpdf0-060, fdac1-021:fdms0-011, fpkt0-084:fadg0-016, fcft0-030:fdms0-011, fcmh0-078:fpkt0-084, fcmh0-078:fdac1-021, fkms0-062:mrcz0-068, mtmr0-066:mdab0-023, mccs0-007:mreb0-061, mdab0-023:mwbt0-001, fjwb0-080:fadg0-016, mabw0-043:mdbb0-041, mpgl0-048:mrcz0-068, mtas1-026:msjs1-002, fcrh0-027:fcmh0-078, mreb0-061:mpdf0-060, mdbb0-041:mtmr0-066, 17 86.0
Accuracy:  0.8023
Average time taken per sample in ms 30.19
Preprocessing MisPredictions:  0
2-gmm_models_scale_test_VidTIMIT/
Speaker Identified Successfully


# Combined dataset

In [83]:
import shutil
import os

directory1 = "2-gmm_models_scale_test_dataset"
directory2 = "2-gmm_models_scale_test_VidTIMIT"
directory_dest = "2-gmm_models_scale_test_combined"

# Copy files from directory1 to directory_dest
for filename in os.listdir(directory1):
    src = os.path.join(directory1, filename)
    dst = os.path.join(directory_dest, filename)
    shutil.copy(src, dst)

# Copy files from directory2 to directory_dest
for filename in os.listdir(directory2):
    src = os.path.join(directory2, filename)
    dst = os.path.join(directory_dest, filename)
    shutil.copy(src, dst)


In [84]:
test = "normalized"
subset = "testing_same"

# test = "normalized"
# subset = "testing_different"

source = f"../../dataset/{test}_dataset/{subset}/"
source2 = f"../../dataset/{test}_dataset/VidTIMIT_testing/"

#path where training speakers will be saved
modelpath = directory_dest

gmm_files = [os.path.join(modelpath,fname) for fname in 
              os.listdir(modelpath) if fname.endswith('.gmm')]

#Load the Gaussian Models
models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
print("Loaded Models: ", len(models))
speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
              in gmm_files]

error = 0
total_samples = 0.0

external_class_true_count = 0
# print("Press '1' for checking a single Audio or Press '0' for testing a complete set of audio with Accuracy?")
# take=int(input().strip())

predictions = []
gender_predictions = []
gender_gt = []
y_true = []
preprocessingMisPredictions = 0
print("Testing the model with all the samples...")
false_predictions = []
test_file = "Testing_audio_Path.txt"       
# file_paths = open(test_file,'r')
file_paths = glob.glob(f"{source}/**/*.wav", recursive=True)
file_paths2 = glob.glob(f"{source2}/**/*.wav", recursive=True)

file_paths = file_paths + file_paths2

# Read the test directory and get the list of test audio files 
avg_time = 0
total_time = 0
for path in file_paths:
    time1 = time.time()   
    total_samples+= 1.0
    path=path.strip()
    # print("Testing Audio : ", path)
    sr,audio = read(path)
    vector   = extract_features(audio,sr)
    # print(vector.shape)
    log_likelihood = np.zeros(len(models)) 
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
    winner=np.argmax(log_likelihood)
    winner_score = log_likelihood[winner]
    predicted_speaker = speakers[winner]
    speaker_label = path.split("/")[-2]
    # print("OK score: ", winner_score)
    predictions.append(predicted_speaker)
    y_true.append(speaker_label)

    if predicted_speaker != speaker_label:
        print(f"{speaker_label}:{predicted_speaker}, ", end="")
        error += 1
        false_predictions.append(f"{speaker_label}:{predicted_speaker} ({path.split('/')[-1]})")

    # time.sleep(1.0)
    sample_time = time.time() - time1
    total_time += sample_time
print (error, total_samples)
accuracy = ((total_samples - error) / total_samples)

print ("Accuracy: ", round(accuracy, 4))
print(f"Average time taken per sample in ms", round((total_time/total_samples)*1000, 2))
print("Preprocessing MisPredictions: ", preprocessingMisPredictions)
# print ("The following Predictions were False :")

print(modelpath)
print ("Speaker Identified Successfully")
with open("scalibility_test.txt", "a") as f:
    print("Combined dataset false predictions (True: False)", file=f)
    for prediction in false_predictions:
        print(prediction, file=f)
    print("Accuracy: ", round(accuracy, 4), file=f)
    print(f"Average time taken per sample in ms", round((total_time/total_samples)*1000, 2), file=f, end="\n\n")



Loaded Models:  49
Testing the model with all the samples...


Renad:Reem, Renad:Reem, fjem0-070:mpdf0-060, fdac1-021:fdms0-011, fpkt0-084:fadg0-016, fcft0-030:fdms0-011, fcmh0-078:fpkt0-084, fcmh0-078:fdac1-021, fkms0-062:mrcz0-068, mtmr0-066:mdab0-023, mccs0-007:mreb0-061, mdab0-023:mwbt0-001, fjwb0-080:fadg0-016, mabw0-043:mdbb0-041, mpgl0-048:mrcz0-068, mtas1-026:msjs1-002, fcrh0-027:fcmh0-078, mreb0-061:mpdf0-060, mdbb0-041:mtmr0-066, 19 206.0
Accuracy:  0.9078
Average time taken per sample in ms 29.21
Preprocessing MisPredictions:  0
2-gmm_models_scale_test_combined
Speaker Identified Successfully
